In [2]:
import pandas as pd
import io
import requests
url = "https://apps.who.int/gho/athena/api/GHO/WHS3_62?filter=COUNTRY:USA&format=csv"
data = requests.get(url).content
csv = pd.read_csv(io.StringIO(data.decode("utf-8")))
print(csv)

        GHO PUBLISHSTATE  YEAR REGION WORLDBANKINCOMEGROUP COUNTRY  \
0   WHS3_62    PUBLISHED  1974    AMR                WB_HI     USA   
1   WHS3_62    PUBLISHED  1975    AMR                WB_HI     USA   
2   WHS3_62    PUBLISHED  1976    AMR                WB_HI     USA   
3   WHS3_62    PUBLISHED  1977    AMR                WB_HI     USA   
4   WHS3_62    PUBLISHED  1978    AMR                WB_HI     USA   
..      ...          ...   ...    ...                  ...     ...   
77  WHS3_62    PUBLISHED  1984    AMR                  NaN     USA   
78  WHS3_62    PUBLISHED  1983    AMR                  NaN     USA   
79  WHS3_62    PUBLISHED  1982    AMR                  NaN     USA   
80  WHS3_62    PUBLISHED  1981    AMR                  NaN     USA   
81  WHS3_62    PUBLISHED  1980    AMR                  NaN     USA   

    Display Value  Numeric  Low  High  Comments  
0           22094  22094.0  NaN   NaN       NaN  
1           24374  24374.0  NaN   NaN       NaN  
2        

In [3]:
import io
import requests
import json
url = "https://apps.who.int/gho/athena/api/GHO/WHS3_62?filter=COUNTRY:USA&format=json"
js = requests.get(url).content
print(js.decode("utf-8"))

{
"copyright": "(c) World Health Organization",
"dataset":
[
{
"label": "CYCU",
"display": "COUNTRY_YEARLY_CORE_UNITS"
}
],
"attribute":
[
{
"label": "DS",
"display": "DS"
},
{
"label": "FIPS",
"display": "FIPS"
},
{
"label": "IOC",
"display": "IOC"
},
{
"label": "ISO2",
"display": "ISO2"
},
{
"label": "ISO",
"display": "ISO"
},
{
"label": "ITU",
"display": "ITU"
},
{
"label": "MARC",
"display": "MARC"
},
{
"label": "WHO",
"display": "WHO"
},
{
"label": "WMO",
"display": "WMO"
},
{
"label": "GEOMETRY",
"display": "GEOMETRY"
},
{
"label": "MORT",
"display": "MORT"
},
{
"label": "LAND_AREA_KMSQ_2012",
"display": "LAND_AREA_KMSQ_2012"
},
{
"label": "LANGUAGES_EN_2012",
"display": "LANGUAGES_EN_2012"
},
{
"label": "WHO_REGION",
"display": "WHO_REGION"
},
{
"label": "WHO_REGION_CODE",
"display": "WHO_REGION_CODE"
},
{
"label": "WORLD_BANK_INCOME_GROUP",
"display": "World Bank income group"
},
{
"label": "WORLD_BANK_INCOME_GROUP_CODE",
"display": "World Bank income group code"
},
{
"label": 